In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras

In [12]:
# Read well log data from CSV file
df = pd.read_csv("combined_freeman_2_3_4.csv")
df.head()

,Depth,GR,Log_ILD,RHOB,NPHI,PHI,PERM,well
0,6308.0,86.730,1.342,2.105,0.396,0.3300,0.2324,freeman_2
1,6308.5,79.598,1.257,2.105,0.381,0.3300,0.2324,freeman_2
2,6309.0,72.180,1.083,2.092,0.365,0.1448,572.4000,freeman_2
3,6309.5,68.811,0.973,2.072,0.373,0.1621,749.9110,freeman_2
4,6310.0,71.912,0.953,2.057,0.372,0.1750,927.9860,freeman_2


In [13]:
# Split data into features (X) and target (y)
X = df.drop(columns=["PERM", 'well'])
y = df["PERM"]

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features using StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
# Define ANN architecture
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=[X_train.shape[1]]),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train model
model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=16)

Epoch 1/50
347/347 [==============================] - 1s 2ms/step - loss: 349919711069913286180864.0000 - val_loss: 199407339867362719432704.0000
Epoch 2/50
347/347 [==============================] - 1s 2ms/step - loss: 349919747098710305144832.0000 - val_loss: 199407339867362719432704.0000
Epoch 3/50
347/347 [==============================] - 1s 2ms/step - loss: 349919747098710305144832.0000 - val_loss: 199407339867362719432704.0000
Epoch 4/50
347/347 [==============================] - 1s 2ms/step - loss: 349919747098710305144832.0000 - val_loss: 199407339867362719432704.0000
Epoch 5/50
347/347 [==============================] - 1s 2ms/step - loss: 349919747098710305144832.0000 - val_loss: 199407339867362719432704.0000
Epoch 6/50
347/347 [==============================] - 1s 2ms/step - loss: 349919747098710305144832.0000 - val_loss: 199407339867362719432704.0000
Epoch 7/50
347/347 [==============================] - 1s 2ms/step - loss: 349919747098710305144832.0000 - val_loss: 19940733

In [15]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

y_pred = model.predict(X_test)

# y_true: true y values, y_pred: predicted y values
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MSE: {:.2f}".format(mse))
print("RMSE: {:.2f}".format(rmse))
print("R2 Score: {:.2f}".format(r2))


93/93 [==============================] - 0s 1ms/step
MSE: 9573341465224255897600.00
RMSE: 97843453870.07
R2 Score: -0.00


In [8]:
# Reshape X_train and X_test into 2D images
n_samples_train, n_features_train = X_train.shape
n_samples_test, n_features_test = X_test.shape
image_shape = (1, int(n_features_train), 1)
X_train_img = np.reshape(X_train, (n_samples_train, *image_shape))
X_test_img = np.reshape(X_test, (n_samples_test, *image_shape))

# Define CNN architecture
model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=image_shape),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train model
model.fit(X_train_img, y_train, validation_split=0.2, epochs=50, batch_size=16)

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d. Consider increasing the input size. Received input shape [None, 1, 6, 1] which would produce output shape with a zero or negative value in a dimension.

In [9]:
def calculate_porosity_from_density(density, matrix_density, fluid_density):
    """
    Calculates porosity from density logs given matrix and fluid density.
    Args:
        density (float or numpy array): density log value(s) in g/cc
        matrix_density (float): matrix density in g/cc
        fluid_density (float): fluid density in g/cc
    Returns:
        float or numpy array: porosity values
    """
    porosity = (matrix_density - density) / (matrix_density - fluid_density)
    return porosity


In [11]:
d = calculate_porosity_from_density(2.105, 2.26, 1.1)
d

0.1336206896551723